## ЛУ 11: Невронна мрежа в numpy

В това упражнение ще програмираме наша собствена невронна мрежа. Тя ще е относително проста, но ще е напълно способна да бъде тренирана и да решава разнообразни задачи. За целта ще използваме единствено методи от библиотеката numpy. 

In [1]:
import numpy as np

### Теория

### Създаване на нетрениран модел

Първата стъпка ще е да създадем шаблонен модел. За целта първо трябва да дефинираме формата на модела си, а после да създадем произволни масиви числа за теглата (weights) и отклоненията (biases) за всеки отделен слой (layer):

In [2]:
# Ще създадем мрежа с четири входни променливи и две изходни
# (за пример ще ползваме задачата за класифициране на ириси в три категории според четири признака).
input_size = 4
output_size = 3

# Ще имаме два скрити слоя, всеки с 40 неврона.
hidden_layer_sizes = [40, 40]

# Вкарваме всичките си променливи в списък.
all_sizes = [input_size] + hidden_layer_sizes + [output_size]

# Създаваме набор от кортежи (tuples), които описват измеренията на всеки слой (вход и изход).
layer_dimensions = [
    (all_sizes[i-1], all_sizes[i])
    for i in range(1, len(all_sizes))
]

layer_dimensions

[(4, 40), (40, 40), (40, 3)]

Тъй като ще използваме кода си многократно, ще дефинираме функции, които да ни помагат. Първоначалните стойности могат да са произволни (в случая с нормална дистрибуция) или просто нули.

In [3]:
def generate_weights( output_dim, input_dim, generate='randn'):

    if generate == 'randn':
        W = np.random.randn(output_dim, input_dim) * 0.1
    
    if generate == 'zeros':
        W = np.zeros(shape=(output_dim, input_dim)) * 0.1

    return W

In [4]:
def generate_biases(output_dim, generate='randn'):

    if generate == 'randn':
        b = np.random.randn(output_dim, 1) * 0.1
    
    if generate == 'zeros':
        b = np.zeros(shape=(output_dim, 1)) * 0.1

    return b

Готови сме да създадем функция, която генерира основата на цялата ни невронна мрежа и я превръща в речник:

In [5]:
def generate_layers(nn_architecture, seed = 42):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, dimensions in enumerate(nn_architecture):
        layer_idx = idx + 1
        
        params_values['W' + str(layer_idx)] = generate_weights(dimensions[1], dimensions[0])
        params_values['b' + str(layer_idx)] = generate_biases(dimensions[1])
        
    return params_values

In [6]:
nn = generate_layers(nn_architecture=layer_dimensions)
nn_shapes = {param: array.shape for param, array in nn.items()}
nn_shapes

{'W1': (40, 4),
 'b1': (40, 1),
 'W2': (40, 40),
 'b2': (40, 1),
 'W3': (3, 40),
 'b3': (3, 1)}

### Изчисляване на слоеве
 Следващата стъпка е да напишем функция, която изчислява формулата за всеки неврон спроед входните данни. Формулата за изчисляване на резултатите от всички неврони е точковият продукт на теглата и активациите на входния слой плюс отклоненията. Накрая трябва да приложим активационната функция върху резутлатите от слоя и получаваме активациите. Така, имаме формулите 
 ![image.png](https://miro.medium.com/v2/resize:fit:640/format:webp/1*-1x2XaJPJXR2FsP1DHG0Vg.gif), където:

 - i е номерът на слоя (т.е. i-1 е предходният слой);
 - z е резултатите на невроните в слоя;
 - W са теглата на слоя;
 - a е резултатът от активациите;
 - b са отклоненията;
 - g е активационната функция.

За активационна функция ще използваме сигмоида, която се изчислява с формулата 
![sigmoid.png](https://www.gstatic.com/education/formulas2/553212783/en/sigmoid_function.svg), където e е числото на Ойлер, 2.71828, а деривативър ѝ е S(x)(1-S(x)). Нека първо дефинираме активацията и дериватива ѝ:


In [7]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

Сега можем да напишем функция за изчисляване на даден слой, както и функция за изчисляване на цялата невронна мрежа:

In [8]:
def single_layer_forward_propagation(A_prev, W_curr, b_curr):
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    A_curr = sigmoid(Z_curr)
    return A_curr, Z_curr

In [9]:
def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, dimensions in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        # print('Layer ',layer_idx)

        W_curr = params_values["W" + str(layer_idx)]
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr)

        # print(A_prev)
        
        # memory съдържа резултатите от изчисленията на различните слоеве
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    return A_curr, memory 

Нека тестваме изходния продукт на необучената ни мрежа:

In [10]:
test_X = np.array([5.3, 6.3, .8, 2.5]).reshape((4, 1)) # примерни входни стойности
Y, memory = full_forward_propagation(test_X, nn, layer_dimensions)

In [11]:
Y

array([[0.48290319],
       [0.5512508 ],
       [0.55008964]])

Виждаме, че произволно генерираните тегла и отклонения ни връщат почти еднакви стойности за дадени входни данни. Начинът, по който ще обучим мрежата си е като изчислим общата грешка (загуба) за всеки слой и нагласим параметрите му така, че по-добре да предвиждат в бъдеще. В случая ще започнем с изходния слой и ще изчислим разликата между y и ŷ, т.е. получения и желания резултат. В случая вторият ще изглежда като [0, 1, 0], [1, 0, 0] или [0, 0, 1] (фиктивни променливи, които отговарят на всяка категория цвете).

Тази разлика ще получим като изчислим деривативите на всяка стъпка от предното движение на мрежата и ги запишем в речника grad_values. Ще използваме следните формули:

![image.png](https://miro.medium.com/v2/resize:fit:536/format:webp/1*FZ4slpsaH_U0YYhaSRqUEQ.gif), където dA е деривативът на A и входните тежести и стойности на следващия слой, dZ е деривативът на активацията между Z и А и т.н. Този метод за оптимизация на алгоритъма се нарича снижаване на градиента (Gradient Descent).

Изчисленията на този етап са най-сложни за разбиране, затова засега нека приложим формулите, а логиката им ще разберем с повече опит с невронните мрежи.

In [12]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev):
    m = A_prev.shape[1]
    
    dZ_curr = sigmoid_backward(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m #T е съкращение за transpose()
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

In [13]:
def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
   
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat))
    
    for layer_idx_prev in reversed(range(len(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1 
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    
    return grads_values

Последната стъпа е да нагодим съществуващите тегла и отклонения спрямо резултатите в речника grad_values.

In [14]:
def update(params_values, grads_values, nn_architecture, learning_rate):
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx+1

        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values

Остава само да напишем функция, която да изпълнява многократно вече дефинираните ни функции, за да обучи напълно дадена мрежа.

In [15]:
def train(X, Y, nn_architecture, epochs, learning_rate):
    
    params_values = generate_layers(nn_architecture, 42)
    
    for i in range(epochs):

        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
    return params_values

Нека опитаме да тренираме мрежата си върху вече познатия ни набор данни ирис.

In [16]:
from seaborn import load_dataset
iris = load_dataset('iris')

In [17]:
X = iris.iloc[:,:4].values
X.shape

(150, 4)

In [18]:
import pandas as pd
Y = pd.get_dummies(iris['species']).values
Y.shape

(150, 3)

Преди да въведем X и Y, трябва да ги транспонираме, за да получим правилни точкови продукти.

In [19]:
trained_nn = train(
    X.T,
    Y.T,
    layer_dimensions,
    epochs = 1000,
    learning_rate = .5
)

In [20]:
Y_pred, memory = full_forward_propagation(X.T, trained_nn, layer_dimensions)
Y_pred.T[:5] # транспонираме отново изходния си продукт

array([[9.97538161e-01, 3.02238422e-03, 4.73647580e-09],
       [9.97167462e-01, 3.52367542e-03, 5.41789218e-09],
       [9.97335201e-01, 3.29387446e-03, 5.11572422e-09],
       [9.96960699e-01, 3.79735165e-03, 5.79828928e-09],
       [9.97557441e-01, 2.99483218e-03, 4.70239752e-09]])

Отава да изчислим най-високата от трите стойности за всеки ред на Y_pred. Това става чрез np.argmax. 

In [21]:
Y_pred_categories = Y_pred.argmax(axis=0)
Y_pred_categories

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

Вече имаме по една стойност на ред, и формата вече ни напомня на разпределението в ирис. Остава само наложим правилните етикети:

In [22]:
structured_predictions = pd.Series(Y_pred_categories).map(dict(zip([0,1,2],iris.species.unique())))
structured_predictions.head()

0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
dtype: object

Нека добавим предвидените стойности в рамката данни:

In [23]:
iris['predicted'] = structured_predictions
iris['correct'] = iris.species == iris.predicted

In [24]:
iris.correct.value_counts()

True     146
False      4
Name: correct, dtype: int64

In [25]:
success_rate = 146/150
success_rate

0.9733333333333334

Успяхме да напишем и обучим невронна мрежа, която познава цветята от рода ирис с 97% точност! Поздравления!

### Задача

Използвайте кода, който написахме, за да обучите невронна мрежа за класифициране върху друг набор данни. (Започнете като измислите подходящи независими и зависими променливи!)

Допълнителни задачи:

 - Ако имате повече опит с програмиране, опитайте се да превърнете невронната ни мрежа в обектно-ориентиран клас.
 - Експериментирайте с различни скорости на обучение и различни параметри за слоевете и вижте колко добре се справя мрежата ви в различни условия.